modified from the unbond-generics example on hackage

On first run `./build.sh` will install dependencies specified in the cabal file into the `pwd` directory,
but import will fail becuase the running kernel would be still oblivious to the newly installed pakcages. 
Once you restart the kernel, the `./build.sh` won't do anyting because everything is to date
and the newly loaded kernel should be able to see the newly installed package modules. 

In [5]:
:!./build.sh

ihaskell-pwd-0.1.0.0: build (exe)
Preprocessing executable 'ihaskell-pwd' for ihaskell-pwd-0.1.0.0..
Building executable 'ihaskell-pwd' for ihaskell-pwd-0.1.0.0..
ihaskell-pwd-0.1.0.0: copy/register
Installing executable ihaskell-pwd in /home/jovyan/pwd/.stack-work/install/x86_64-linux/lts-13.26/8.6.5/bin

In [6]:
{-# LANGUAGE DeriveDataTypeable
           , DeriveGeneric
           , MultiParamTypeClasses #-}

import Unbound.Generics.LocallyNameless
import GHC.Generics (Generic)
import Data.Typeable (Typeable)

In [7]:
-- | Names for expressions
type Nm = Name Exp

-- | Expressions
data Exp = Var Nm            -- ^ variables
         | Lam (Bind Nm Exp) -- ^ lambdas bind a variable within a body expression
         | App Exp Exp       -- ^ application
          deriving (Show, Generic, Typeable)

-- Automatically construct alpha equiv, free vars, and binding operations.
instance Alpha Exp

-- semi-automatically implement capture avoiding substitution
instance Subst Exp Exp where
  -- `isvar` identifies the variable case in your AST.
  isvar (Var x) = Just (SubstName x)
  isvar _       = Nothing

In [16]:
import Unbound.Generics.LocallyNameless.Unsafe
:type unsafeUnbind

unsafeUnbind :: forall p t. (Alpha p, Alpha t) => Bind p t -> (p, t)

In [31]:
import IHaskell.Display

dpExp e = html["code"++ppExp e,"<code>"]

ppExp (Var x) = show x
ppExp (Lam bnd) = "λ" ++ show x ++ "." ++ ppExp t
  where (x, t) = unsafeUnbind bnd
ppExp (App t s) = ppt t ++ " " ++ pps s
  where
  ppt t@(Lam{}) = paren (ppExp t)
  ppt t         = ppExp t
  pps s@(Var{}) = ppExp s
  pps s         = paren (ppExp s)

paren s = "("++s++")"

: 

In [23]:
-- call-by-value evaluation
-- evaluation takes an expression and returns a value while using a source of fresh names
eval :: Exp -> FreshM Exp
eval (Var x)     = fail $ "unbound variable " ++ show x
eval e@(Lam {})  = return e
eval (App e1 e2) = do
  v1 <- eval e1
  v2 <- eval e2
  case v1 of
   (Lam bnd) -> do
     -- open the lambda by picking a fresh name for the bound variable x in body
     (x, body) <- unbind bnd
     eval (subst x v2 body)
   _ -> fail "application of non-lambda"

In [24]:
x = s2n "x"
y = s2n "y"
e = Lam $ bind x (Lam $ bind y (App (Var y) (Var x)))

In [25]:
e
ppExp e

Lam (<x> Lam (<y> App (Var 0@0) (Var 1@0)))

"\955x.\955y.y x"

In [12]:
runFreshM $ eval (App (App e e) e)

Lam (<y> App (Var 0@0) (Lam (<x> Lam (<y> App (Var 0@0) (Var 1@0)))))